# **LOGISTIC REGRESSION**

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # "0" o "1"

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
from utils import *

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, confusion_matrix

In [ ]:
# -----------------------
# Model building function
# -----------------------

def build_model(C=1.0, penalty="l2", solver="lbfgs"):
    """
    Builds a scikit-learn Pipeline with TF-IDF vectorization and Logistic Regression classifier.

    Args:
        C (float): Inverse of regularization strength for Logistic Regression.
        penalty (str): Regularization type for Logistic Regression
        solver (str): Solver to use for Logistic Regression.

    Returns:
        Pipeline: A scikit-learn Pipeline object (TF-IDF + Logistic Regression).
    """
    
    return Pipeline([
        ('tfidf', TfidfVectorizer(  # TD-IDF vectorization
            max_features=5000,      # limit to top 5000 features
            ngram_range=(1, 2),     # unigrams + bigrams
            stop_words="english"    # remove English stop words
        )),
        ('clf', LogisticRegression( # Logistic Regression classifier
            C=C,
            penalty=penalty if penalty != "none" else None,
            solver=solver,
            max_iter=1000,
            random_state=42,
        ))
    ])

## VERSION 1: Dataset (Simple)

In [5]:
# --------------------------------
# Fine-tuning on multiple datasets
# --------------------------------

datasets_df = data_loading() # load datasets

datasets = {name: split_dataset(df) for name, df in datasets_df.items()} # split all datasets in train/val/test
model = build_model() # initialize model

results = {}

# sequential training
for i, (name, data) in enumerate(datasets.items()):
    print(f"\n=== Phase {i+1}: Training/Fine-tuning on {name} ===")
    
    X_train, y_train = data["train"]
    X_val, y_val = data["val"]
    X_test, y_test = data["test"]

    model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val])) # fine-tune on train + val

    y_pred = model.predict(X_test)
    print(f"Classification Report after {name}:")
    print(classification_report(y_test, y_pred))
    print(f"Confusion Matrix after {name}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"\nWeighted F1-score after {name}:", f1_score(y_test, y_pred, average="weighted"))


    # evaluation on all datasets
    print("\n--- Evaluation on all datasets ---")
    results[name] = {}
    for test_name, test_data in datasets.items(): # for each dataset
        X_te, y_te = test_data["test"]
        preds = model.predict(X_te)
        f1 = f1_score(y_te, preds, average="weighted")
        results[name][test_name] = f1
        print(f"Evaluation on {test_name}: Weighted F1 = {f1:.4f}")


/home/n.emmolo/progetto_tesi/src/utils.py:162: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  dfKaggleMeg['date'] = pd.to_datetime(dfKaggleMeg['date'], errors='coerce') # convert date column to datetime, coerce errors to NaT



=== Phase 1: Training/Fine-tuning on Celebrity ===
Classification Report after Celebrity:
              precision    recall  f1-score   support

           0       0.65      0.84      0.73        50
           1       0.77      0.54      0.64        50

    accuracy                           0.69       100
   macro avg       0.71      0.69      0.68       100
weighted avg       0.71      0.69      0.68       100

Confusion Matrix after Celebrity:
[[42  8]
 [23 27]]

Weighted F1-score after Celebrity: 0.6828644501278772

--- Evaluation on all datasets ---
Evaluation on Celebrity: Weighted F1 = 0.6829
Evaluation on CIDII: Weighted F1 = 0.5762
Evaluation on FaKES: Weighted F1 = 0.4241
Evaluation on FakeVsSatire: Weighted F1 = 0.3143
Evaluation on Horne: Weighted F1 = 0.5014
Evaluation on Infodemic: Weighted F1 = 0.5466
Evaluation on ISOT: Weighted F1 = 0.3799
Evaluation on Kaggle_clement: Weighted F1 = 0.4266
Evaluation on Kaggle_meg: Weighted F1 = 0.8955
Evaluation on LIAR_PLUS: Weighte

In [6]:
# ---------------
# Results summary
# ---------------

print("\n=== Results Summary ===")
for name, res in results.items():
    print(f"\nResults after training on {name}:")
    for test_name, f1 in res.items():
        print(f"  Test on {test_name}: Weighted F1 = {f1:.4f}")


=== Results Summary ===

Results after training on Celebrity:
  Test on Celebrity: Weighted F1 = 0.6829
  Test on CIDII: Weighted F1 = 0.5762
  Test on FaKES: Weighted F1 = 0.4241
  Test on FakeVsSatire: Weighted F1 = 0.3143
  Test on Horne: Weighted F1 = 0.5014
  Test on Infodemic: Weighted F1 = 0.5466
  Test on ISOT: Weighted F1 = 0.3799
  Test on Kaggle_clement: Weighted F1 = 0.4266
  Test on Kaggle_meg: Weighted F1 = 0.8955
  Test on LIAR_PLUS: Weighted F1 = 0.5316
  Test on Politifact: Weighted F1 = 0.6230
  Test on Unipi_NDF: Weighted F1 = 0.5003

Results after training on CIDII:
  Test on Celebrity: Weighted F1 = 0.4415
  Test on CIDII: Weighted F1 = 0.8724
  Test on FaKES: Weighted F1 = 0.3764
  Test on FakeVsSatire: Weighted F1 = 0.3192
  Test on Horne: Weighted F1 = 0.5775
  Test on Infodemic: Weighted F1 = 0.4860
  Test on ISOT: Weighted F1 = 0.4455
  Test on Kaggle_clement: Weighted F1 = 0.5040
  Test on Kaggle_meg: Weighted F1 = 0.9043
  Test on LIAR_PLUS: Weighted F1 = 0

## VERSION 2: Dataset by Topic

In [ ]:
# -------------------------------
# Fine-tuning on Dataset by Topic
# -------------------------------

dataset_df = data_by_topic()

datasets = {topic: split_dataset(df) for topic, df in dataset_df.items()} # split all datasets in train/val/test
model = build_model() # initialize model

results = {}

# sequential training
for i, (topic, data) in enumerate(datasets.items()):
    print(f"\n=== Phase {i+1}: Training/Fine-tuning on topic: {topic} ===")
    
    X_train, y_train = data["train"]
    X_val, y_val = data["val"]
    X_test, y_test = data["test"]

    model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val])) # fine-tune on train + val

    y_pred = model.predict(X_test)
    print(f"Classification Report after topic {topic}:")
    print(classification_report(y_test, y_pred))
    print(f"Confusion Matrix after topic {topic}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"\nWeighted F1-score after topic {topic}:", f1_score(y_test, y_pred, average="weighted"))


    # evaluation on all topics
    print("\n--- Evaluation on all topics ---")
    results[topic] = {}
    for test_topic, test_data in datasets.items(): # for each topic
        X_te, y_te = test_data["test"]
        preds = model.predict(X_te)
        f1 = f1_score(y_te, preds, average="weighted")
        results[topic][test_topic] = f1
        print(f"Evaluation on topic {test_topic}: Weighted F1 = {f1:.4f}")

/home/n.emmolo/progetto_tesi/src/utils.py:162: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  dfKaggleMeg['date'] = pd.to_datetime(dfKaggleMeg['date'], errors='coerce') # convert date column to datetime, coerce errors to NaT



=== Phase 1: Training/Fine-tuning on topic: politics ===
Classification Report after topic politics:
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93     10094
         1.0       0.93      0.93      0.93      9402

    accuracy                           0.93     19496
   macro avg       0.93      0.93      0.93     19496
weighted avg       0.93      0.93      0.93     19496

Confusion Matrix after topic politics:
[[9416  678]
 [ 643 8759]]

Weighted F1-score after topic politics: 0.9322466151502965

--- Evaluation on all topics ---
Evaluation on topic politics: Weighted F1 = 0.9322
Evaluation on topic general: Weighted F1 = 0.2212
Evaluation on topic covid: Weighted F1 = 0.3626
Evaluation on topic syria: Weighted F1 = 0.4746
Evaluation on topic islam: Weighted F1 = 0.4866
Evaluation on topic notredame: Weighted F1 = 0.2855
Evaluation on topic gossip: Weighted F1 = 0.3711

=== Phase 2: Training/Fine-tuning on topic: general ===
Classifi

In [ ]:
# ---------------
# Results summary
# ---------------

print("\n=== Results Summary ===")
for topic, res in results.items():
    print(f"\nResults after training on topic {topic}:")
    for test_topic, f1 in res.items():
        print(f"  Test on topic {test_topic}: Weighted F1 = {f1:.4f}")


=== Results Summary ===

Results after training on topic politics:
  Test on topic politics: Weighted F1 = 0.9322
  Test on topic general: Weighted F1 = 0.2212
  Test on topic covid: Weighted F1 = 0.3626
  Test on topic syria: Weighted F1 = 0.4746
  Test on topic islam: Weighted F1 = 0.4866
  Test on topic notredame: Weighted F1 = 0.2855
  Test on topic gossip: Weighted F1 = 0.3711

Results after training on topic general:
  Test on topic politics: Weighted F1 = 0.3532
  Test on topic general: Weighted F1 = 0.9690
  Test on topic covid: Weighted F1 = 0.3600
  Test on topic syria: Weighted F1 = 0.3633
  Test on topic islam: Weighted F1 = 0.4333
  Test on topic notredame: Weighted F1 = 0.4654
  Test on topic gossip: Weighted F1 = 0.3333

Results after training on topic covid:
  Test on topic politics: Weighted F1 = 0.3696
  Test on topic general: Weighted F1 = 0.1997
  Test on topic covid: Weighted F1 = 0.9399
  Test on topic syria: Weighted F1 = 0.3363
  Test on topic islam: Weighted F

## VERSION 3: Dataset by Date

In [ ]:
dataset_df = data_by_date()

for date, df in dataset_df.items():
    print(f"Date: {date}, Number of samples: {len(df)}")

/home/n.emmolo/progetto_tesi/src/utils.py:162: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  dfKaggleMeg['date'] = pd.to_datetime(dfKaggleMeg['date'], errors='coerce') # convert date column to datetime, coerce errors to NaT


{'2011-2013': 55, 2014: 114, 2015: 84, 2016: 49687, 2017: 16657, 2020: 10559}

In [ ]:
# ------------------------------
# Fine-tuning on Dataset by Date
# ------------------------------

dataset_df = data_by_date()

datasets = {date: split_dataset(df) for date, df in dataset_df.items()} # split all datasets in train/val/test
model = build_model() # initialize model

results = {}

# sequential training
for i, (date, data) in enumerate(datasets.items()):
    print(f"\n=== Phase {i+1}: Training/Fine-tuning on date: {date} ===")
    
    X_train, y_train = data["train"]
    X_val, y_val = data["val"]
    X_test, y_test = data["test"]

    model.fit(pd.concat([X_train, X_val]), pd.concat([y_train, y_val])) # fine-tune on train + val

    y_pred = model.predict(X_test)
    print(f"Classification Report after date {date}:")
    print(classification_report(y_test, y_pred))
    print(f"Confusion Matrix after date {date}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"\nWeighted F1-score after date {date}:", f1_score(y_test, y_pred, average="weighted"))


    # evaluation on all dates
    print("\n--- Evaluation on all dates ---")
    results[date] = {}
    for test_date, test_data in datasets.items(): # for each date
        X_te, y_te = test_data["test"]
        preds = model.predict(X_te)
        f1 = f1_score(y_te, preds, average="weighted")
        results[date][test_date] = f1
        print(f"Evaluation on date {test_date}: Weighted F1 = {f1:.4f}")

/home/n.emmolo/progetto_tesi/src/utils.py:162: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  dfKaggleMeg['date'] = pd.to_datetime(dfKaggleMeg['date'], errors='coerce') # convert date column to datetime, coerce errors to NaT



=== Phase 1: Training/Fine-tuning on date: 2011-2013 ===
Classification Report after date 2011-2013:
              precision    recall  f1-score   support

         0.0       1.00      0.20      0.33         5
         1.0       0.60      1.00      0.75         6

    accuracy                           0.64        11
   macro avg       0.80      0.60      0.54        11
weighted avg       0.78      0.64      0.56        11

Confusion Matrix after date 2011-2013:
[[1 4]
 [0 6]]

Weighted F1-score after date 2011-2013: 0.5606060606060606

--- Evaluation on all dates ---
Evaluation on date 2011-2013: Weighted F1 = 0.5606
Evaluation on date 2014: Weighted F1 = 0.2690
Evaluation on date 2015: Weighted F1 = 0.3665
Evaluation on date 2016: Weighted F1 = 0.3004
Evaluation on date 2017: Weighted F1 = 0.0030
Evaluation on date 2020: Weighted F1 = 0.3070

=== Phase 2: Training/Fine-tuning on date: 2014 ===
Classification Report after date 2014:
              precision    recall  f1-score   suppo

/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

Evaluation on date 2016: Weighted F1 = 0.3174
Evaluation on date 2017: Weighted F1 = 0.0667
Evaluation on date 2020: Weighted F1 = 0.3219

=== Phase 4: Training/Fine-tuning on date: 2016 ===
Classification Report after date 2016:
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      5306
         1.0       0.98      0.98      0.98      4632

    accuracy                           0.98      9938
   macro avg       0.98      0.98      0.98      9938
weighted avg       0.98      0.98      0.98      9938

Confusion Matrix after date 2016:
[[5232   74]
 [  80 4552]]

Weighted F1-score after date 2016: 0.9845032811664566

--- Evaluation on all dates ---
Evaluation on date 2011-2013: Weighted F1 = 0.3961
Evaluation on date 2014: Weighted F1 = 0.4214
Evaluation on date 2015: Weighted F1 = 0.3827
Evaluation on date 2016: Weighted F1 = 0.9845
Evaluation on date 2017: Weighted F1 = 0.9915
Evaluation on date 2020: Weighted F1 = 0.3092

=== Phase 5:

/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/n.emmolo/miniconda3/envs/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

Evaluation on date 2016: Weighted F1 = 0.3719
Evaluation on date 2017: Weighted F1 = 0.9937
Evaluation on date 2020: Weighted F1 = 0.3600

=== Phase 6: Training/Fine-tuning on date: 2020 ===
Classification Report after date 2020:
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      1106
         1.0       0.92      0.93      0.93      1006

    accuracy                           0.93      2112
   macro avg       0.93      0.93      0.93      2112
weighted avg       0.93      0.93      0.93      2112

Confusion Matrix after date 2020:
[[1026   80]
 [  68  938]]

Weighted F1-score after date 2020: 0.9299408611809796

--- Evaluation on all dates ---
Evaluation on date 2011-2013: Weighted F1 = 0.3409
Evaluation on date 2014: Weighted F1 = 0.3991
Evaluation on date 2015: Weighted F1 = 0.3388
Evaluation on date 2016: Weighted F1 = 0.3473
Evaluation on date 2017: Weighted F1 = 0.1014
Evaluation on date 2020: Weighted F1 = 0.9299


In [ ]:
# ---------------
# Results summary
# ---------------

print("\n=== Results Summary ===")
for date, res in results.items():
    print(f"\nResults after training on date {date}:")
    for test_date, f1 in res.items():
        print(f"  Test on date {test_date}: Weighted F1 = {f1:.4f}")


=== Results Summary ===

Results after training on date 2011-2013:
  Test on date 2011-2013: Weighted F1 = 0.5606
  Test on date 2014: Weighted F1 = 0.2690
  Test on date 2015: Weighted F1 = 0.3665
  Test on date 2016: Weighted F1 = 0.3004
  Test on date 2017: Weighted F1 = 0.0030
  Test on date 2020: Weighted F1 = 0.3070

Results after training on date 2014:
  Test on date 2011-2013: Weighted F1 = 0.6303
  Test on date 2014: Weighted F1 = 0.4283
  Test on date 2015: Weighted F1 = 0.4938
  Test on date 2016: Weighted F1 = 0.3604
  Test on date 2017: Weighted F1 = 0.8698
  Test on date 2020: Weighted F1 = 0.3937

Results after training on date 2015:
  Test on date 2011-2013: Weighted F1 = 0.3409
  Test on date 2014: Weighted F1 = 0.3095
  Test on date 2015: Weighted F1 = 0.3665
  Test on date 2016: Weighted F1 = 0.3174
  Test on date 2017: Weighted F1 = 0.0667
  Test on date 2020: Weighted F1 = 0.3219

Results after training on date 2016:
  Test on date 2011-2013: Weighted F1 = 0.3961
